<a href="https://colab.research.google.com/github/kojikokojiko/portfoilio/blob/master/M5/val_pickle3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!ls

 c_weight.pickle		   'val_pickle3 '
'lags features'			    予測ノーマル
 pickle1.ipynb			    予測CA　いわ
 pickle2.ipynb			   '予測CA　いわ のコピー'
 pickle3.ipynb			    予測.ipynb
 sample_submission変換		   '予測TX ryushi'
 shop_id分け+mean_enc上書き.ipynb  '予測WI takemi'
'val_pickle1 '			    調整.ipynb
'val_pickle2 '


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%cd /content/drive/My Drive/kaggle/M5/iwase

/content/drive/.shortcut-targets-by-id/18e73-MFsSq-bfNImjjh-axhWYXPQE-7c/kaggle/M5/iwase


In [0]:

# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

from math import ceil

from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings('ignore')

In [0]:
grid_df=pd.read_pickle('../iwase_output/val/val_grid_part_1.pkl')

In [0]:
# prices_df = pd.read_csv('../input/m5-forecasting-accuracy/sell_prices.csv')
calendar_df = pd.read_csv('../input/calendar.csv')

In [0]:
calendar_df

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [0]:
#################################################################################
TARGET = 'sales'         # Our main target
END_TRAIN = 1913        # Last day in train set
MAIN_INDEX = ['id','d']  # We can identify item by these columns

In [0]:
grid_df=grid_df[MAIN_INDEX]

In [0]:
icols = ['date',
         'd',
         'event_name_1',
         'event_type_1',
         'event_name_2',
         'event_type_2',
         'snap_CA',
         'snap_TX',
         'snap_WI']


In [0]:
grid_df=grid_df.merge(calendar_df[icols],on='d',how='left')

In [0]:
icols = ['event_name_1',
         'event_type_1',
         'event_name_2',
         'event_type_2',
         'snap_CA',
         'snap_TX',
         'snap_WI']
for col in icols:
    grid_df[col] = grid_df[col].astype('category')

In [0]:
grid_df['date']=pd.to_datetime(grid_df['date'])

In [0]:
grid_df['tm_d'] = grid_df['date'].dt.day.astype(np.int8)
grid_df['tm_w'] = grid_df['date'].dt.week.astype(np.int8)
grid_df['tm_m'] = grid_df['date'].dt.month.astype(np.int8)
grid_df['tm_y'] = grid_df['date'].dt.year
grid_df['tm_y'] = (grid_df['tm_y'] - grid_df['tm_y'].min()).astype(np.int8)
grid_df['tm_wm'] = grid_df['tm_d'].apply(lambda x: ceil(x/7)).astype(np.int8)

grid_df['tm_dw'] = grid_df['date'].dt.dayofweek.astype(np.int8)
grid_df['tm_w_end'] = (grid_df['tm_dw']>=5).astype(np.int8)

In [0]:
del grid_df['date']


In [0]:
# grid_df=pd.read_pickle('../iwase_output/grid_part_3.pkl')

In [0]:
grid_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46881677 entries, 0 to 46881676
Data columns (total 16 columns):
 #   Column        Dtype   
---  ------        -----   
 0   id            category
 1   d             object  
 2   event_name_1  category
 3   event_type_1  category
 4   event_name_2  category
 5   event_type_2  category
 6   snap_CA       category
 7   snap_TX       category
 8   snap_WI       category
 9   tm_d          int8    
 10  tm_w          int8    
 11  tm_m          int8    
 12  tm_y          int8    
 13  tm_wm         int8    
 14  tm_dw         int8    
 15  tm_w_end      int8    
dtypes: category(8), int8(7), object(1)
memory usage: 1.4+ GB


In [0]:
encode_columns=["event_name_1",	"event_type_1",	"event_name_2",	"event_type_2"]

In [0]:
grid_df

,id,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,tm_d,tm_w,tm_m,tm_y,tm_wm,tm_dw,tm_w_end
0,HOBBIES_1_008_CA_1_validation,d_1,NaN,NaN,NaN,NaN,0,0,0,29,4,1,0,5,5,1
1,HOBBIES_1_009_CA_1_validation,d_1,NaN,NaN,NaN,NaN,0,0,0,29,4,1,0,5,5,1
2,HOBBIES_1_010_CA_1_validation,d_1,NaN,NaN,NaN,NaN,0,0,0,29,4,1,0,5,5,1
3,HOBBIES_1_012_CA_1_validation,d_1,NaN,NaN,NaN,NaN,0,0,0,29,4,1,0,5,5,1
4,HOBBIES_1_015_CA_1_validation,d_1,NaN,NaN,NaN,NaN,0,0,0,29,4,1,0,5,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46881672,FOODS_3_823_WI_3_validation,d_1941,NaN,NaN,NaN,NaN,0,0,0,22,20,5,5,4,6,1
46881673,FOODS_3_824_WI_3_validation,d_1941,NaN,NaN,NaN,NaN,0,0,0,22,20,5,5,4,6,1
46881674,FOODS_3_825_WI_3_validation,d_1941,NaN,NaN,NaN,NaN,0,0,0,22,20,5,5,4,6,1
46881675,FOODS_3_826_WI_3_validation,d_1941,NaN,NaN,NaN,NaN,0,0,0,22,20,5,5,4,6,1


In [0]:
for column in encode_columns:
  grid_df[column]=grid_df[column].cat.add_categories('XXX').fillna('XXX')
  

In [0]:
grid_df

,id,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,tm_d,tm_w,tm_m,tm_y,tm_wm,tm_dw,tm_w_end
0,HOBBIES_1_008_CA_1_validation,d_1,XXX,XXX,XXX,XXX,0,0,0,29,4,1,0,5,5,1
1,HOBBIES_1_009_CA_1_validation,d_1,XXX,XXX,XXX,XXX,0,0,0,29,4,1,0,5,5,1
2,HOBBIES_1_010_CA_1_validation,d_1,XXX,XXX,XXX,XXX,0,0,0,29,4,1,0,5,5,1
3,HOBBIES_1_012_CA_1_validation,d_1,XXX,XXX,XXX,XXX,0,0,0,29,4,1,0,5,5,1
4,HOBBIES_1_015_CA_1_validation,d_1,XXX,XXX,XXX,XXX,0,0,0,29,4,1,0,5,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46881672,FOODS_3_823_WI_3_validation,d_1941,XXX,XXX,XXX,XXX,0,0,0,22,20,5,5,4,6,1
46881673,FOODS_3_824_WI_3_validation,d_1941,XXX,XXX,XXX,XXX,0,0,0,22,20,5,5,4,6,1
46881674,FOODS_3_825_WI_3_validation,d_1941,XXX,XXX,XXX,XXX,0,0,0,22,20,5,5,4,6,1
46881675,FOODS_3_826_WI_3_validation,d_1941,XXX,XXX,XXX,XXX,0,0,0,22,20,5,5,4,6,1


In [0]:
for column in encode_columns:
  le=LabelEncoder()
  le.fit(grid_df[column])
  grid_df[column]=le.transform(grid_df[column])
  exec('encode_'+ column +'=le.classes_')



In [0]:
encode_event_name_1

array(['Chanukah End', 'Christmas', 'Cinco De Mayo', 'ColumbusDay',
       'Easter', 'Eid al-Fitr', 'EidAlAdha', "Father's day", 'Halloween',
       'IndependenceDay', 'LaborDay', 'LentStart', 'LentWeek2',
       'MartinLutherKingDay', 'MemorialDay', "Mother's day",
       'NBAFinalsEnd', 'NBAFinalsStart', 'NewYear', 'OrthodoxChristmas',
       'OrthodoxEaster', 'Pesach End', 'PresidentsDay', 'Purim End',
       'Ramadan starts', 'StPatricksDay', 'SuperBowl', 'Thanksgiving',
       'ValentinesDay', 'VeteransDay', 'XXX'], dtype=object)

In [0]:
encode_event_name_2

array(['Cinco De Mayo', 'Easter', "Father's day", 'OrthodoxEaster', 'XXX'],
      dtype=object)

In [0]:
encode_event_type_1

array(['Cultural', 'National', 'Religious', 'Sporting', 'XXX'],
      dtype=object)

In [0]:
encode_event_type_2

array(['Cultural', 'Religious', 'XXX'], dtype=object)

In [0]:
grid_df.to_pickle('../iwase_output/val/val_grid_part_3.pkl')

https://www.it-swarm.dev/ja/python/pandas%E3%82%AB%E3%83%86%E3%82%B4%E3%83%AA%E3%83%87%E3%83%BC%E3%82%BF%E3%81%ABnan%E3%82%92%E5%85%A5%E5%8A%9B%E3%81%99%E3%82%8B/1055392575/amp/


categoryがないものをfillnaしようとすると、エラーが起きる


In [0]:
# grid_df=pd.read_pickle('../iwase_output/val/val_grid_part_3.pkl')

In [0]:
# grid_df